
## **Парсим сайт**
1.   Перебираем ссылки, ищем рабочие
2.   Из рабочих ссылок достаём текст задачи и темы задачи
3.   Приводим в нормальный вид полученные данные

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests

Создаем файл с работающими ссылками

In [ ]:
working_links = []

for num in range(0000, 8000):

    url = f"https://www.iloveeconomics.ru/z/{num:04d}"
    response = requests.get(url)

    if response.status_code == 200:
            working_links.append(url)

    print(f"Processed: {num}, Working Links: {len(working_links)}", end="\r")

with open("url_list.txt", "w") as file:
    for link in working_links:
        file.write(str(link) + "\n")

Используя файл с ссылками парсим текст задачи и тему

In [2]:
with open('url_list.txt', "r") as file:
    lines = file.readlines()
lines = [line.strip() for line in lines]

df = pd.DataFrame(columns=["Task", "Theme"])

for url in lines:

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")

        task_items = soup.find_all(class_="fix_width")

        if task_items:
            tasks = []
            themes = []

            for task_item in task_items:
                task_text = task_item.find(class_="tex2jax")
                tasks.append(task_text)

                theme_text = task_item.find(class_="field-item even")
                themes.append(theme_text)

            temp_df = pd.DataFrame({"Task": tasks, "Theme": themes})

            df = pd.concat([df, temp_df], ignore_index=True)

df.dropna(inplace=True)
df.reset_index(drop= True , inplace= True)

приводим полученые данные в читаемый вид, удаляем лишние символы

In [3]:
df.Task = df.Task.astype('str')
df.Theme = df.Theme.astype('str')

df["Task"] = df["Task"].apply(lambda x: re.sub(r"<.*?>", "", x))
df["Theme"] = df["Theme"].apply(lambda x: re.sub(r"<.*?>", "", x))
df["Task"] = df["Task"].apply(lambda x: x.replace("\n", "").replace("\r", ""))

In [4]:
df.to_csv('df.csv') # качаем файл с данными